In [231]:
import os
from scipy import signal
import nibabel as nib
import numpy as np
import argparse
from Unet import Generic_UNet, InitWeights_He
import math
from tqdm import tqdm
import torch
import torch.nn as nn
from common_test_Unet import testatlas as test
from scipy.ndimage.filters import gaussian_filter
from collections import OrderedDict
from numpy.linalg import inv
from utilities import SoftDiceLoss, resize_segmentation, _concat, _concatmodel, one_hot2dist, ComputMetric
from sampling_multiprocess import get_augment_par
from common_test_Unet import pad_nd_image, _compute_steps_for_sliding_window, getallbatch
import torch.nn.functional as F

In [232]:
prostateckpt = '/vol/biomedic3/zl9518/ModelEvaluation/Segment/Prostateckpt/checkpoint.pth.tar'
atlasckpt = '/vol/biomedic3/zl9518/ModelEvaluation/Segment/ATLASckpt/checkpoint.pth.tar'

In [233]:
prostatevalpath = '/vol/biomedic3/zl9518/Prostatedata/datafiletestB/'
prostatetestpath1 = '/vol/biomedic3/zl9518/Prostatedata/datafileA/'
prostatetestpath2 = '/vol/biomedic3/zl9518/Prostatedata/datafileC/'
prostatetestpath3 = '/vol/biomedic3/zl9518/Prostatedata/datafileD/'
prostatetestpath4 = '/vol/biomedic3/zl9518/Prostatedata/datafileE/'
prostatetestpath5 = '/vol/biomedic3/zl9518/Prostatedata/datafileF/'
prostatetestpaths = [prostatetestpath1, prostatetestpath2, prostatetestpath3, prostatetestpath4, prostatetestpath5]
atlasvalpath = '/vol/biomedic2/zl9518/ATLAS/datafilenews/validation/'

In [234]:
savename = 'prostateval'
savedckpt = prostateckpt
patch_size = [64, 64, 32]
testlist = [0]
testprob = [1]
NumsInputChannel = 1
NumsClass = 2
pathname = 'prostate'
DatafileValFoldtr = None
DatafileValFoldts = prostatevalpath

In [235]:
args = {'resume': savedckpt, 'name': savename, 'saveresults': True, 'patch_size': patch_size, 'downsampling': 4,
       'ttalist': testlist, 'ttalistprob': testprob, 'features': 10, 'deepsupervision': True, 'gpu': 0}

In [236]:
torch.cuda.set_device(args['gpu'])
# create model
conv_op = nn.Conv3d
dropout_op = nn.Dropout3d
norm_op = nn.InstanceNorm3d
conv_per_stage = 2
base_num_features = args['features']
args['features'] = base_num_features

norm_op_kwargs = {'eps': 1e-5, 'affine': True}
dropout_op_kwargs = {'p': 0, 'inplace': True}
net_nonlin = nn.LeakyReLU
net_nonlin_kwargs = {'negative_slope': 1e-2, 'inplace': True}
net_num_pool_op_kernel_sizes = []
if args['patch_size'][1] != args['patch_size'][2]:
    net_num_pool_op_kernel_sizes.append([2, 2, 1])
    for kiter in range(0, args['downsampling'] - 1):  # (0,5)
        net_num_pool_op_kernel_sizes.append([2, 2, 2])
else:
    for kiter in range(0, args['downsampling']):  # (0,5)
        net_num_pool_op_kernel_sizes.append([2, 2, 2])
net_conv_kernel_sizes = []
for kiter in range(0, args['downsampling'] + 1):  # (0,6)
    net_conv_kernel_sizes.append([3, 3, 3])

model = Generic_UNet(NumsInputChannel, base_num_features, NumsClass,
                     len(net_num_pool_op_kernel_sizes),
                     conv_per_stage, 2, conv_op, norm_op, norm_op_kwargs, dropout_op,
                     dropout_op_kwargs,
                     net_nonlin, net_nonlin_kwargs, args['deepsupervision'], False, lambda x: x, InitWeights_He(1e-2),
                     net_num_pool_op_kernel_sizes, net_conv_kernel_sizes, False, True, True)
model = model.cuda()
model.eval()

if args['resume']:
    if os.path.isfile(args['resume']):
        print("=> loading checkpoint '{}'".format(args['resume']))
        checkpoint = torch.load(args['resume'], map_location='cuda:' + str(args['gpu']))
        model.load_state_dict(checkpoint['state_dict'])
        print("=> loaded checkpoint '{}' (epoch {})".format(args['resume'], checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(args['resume']))

=> loading checkpoint '/vol/biomedic3/zl9518/ModelEvaluation/Segment/Prostateckpt/checkpoint.pth.tar'
=> loaded checkpoint '/vol/biomedic3/zl9518/ModelEvaluation/Segment/Prostateckpt/checkpoint.pth.tar' (epoch 1000)


In [237]:
def testmap(model, saveresults, name, pathname = None, ImgsegmentSize = [80, 160, 160], deepsupervision=False, 
    DatafileValFoldtr=None, DatafileValFoldts=None, tta=False, ttalist=[0], NumsClass = 2, ttalistprob=[1]):
    
    batch_size = 1
    NumsInputChannel = 1

    DatafiletsFold = DatafileValFoldts
    DatafiletsImgc1 = DatafiletsFold + 'Imgpre-eval.txt'
    DatafiletsLabel = DatafiletsFold + 'seg-eval.txt'
    DatafiletsMask = DatafiletsFold + 'mask-eval.txt'

    Imgfiletsc1 = open(DatafiletsImgc1)
    Imgreadtsc1 = Imgfiletsc1.read().splitlines()
    if os.path.isfile(DatafiletsMask):
        Masktsfile = open(DatafiletsMask)
        Masktsread = Masktsfile.read().splitlines()
        Maskread = Masktsread
    Labeltsfile = open(DatafiletsLabel)
    Labeltsread = Labeltsfile.read().splitlines()

    Imgreadc1 = Imgreadtsc1
    Labelread = Labeltsread
    
    DSClist = []
    
    for numr in range(len(Imgreadc1)):

        Imgnamec1 = Imgreadc1[numr]
        Imgloadc1 = nib.load(Imgnamec1)
        Imgc1 = Imgloadc1.get_fdata()
        
        if os.path.isfile(DatafiletsMask):
            Maskname = Maskread[numr]
            Maskload = nib.load(Maskname)
            roi_mask = Maskload.get_fdata()
        Labelname = Labelread[numr]
        Labelload = nib.load(Labelname)
        gtlabel = Labelload.get_fdata()
        
        Imgc1 = np.float32(Imgc1)

        knamelist = Imgnamec1.split("/")
        kname = knamelist[-1][0:6]

        channels = Imgc1[None, ...]

        from common_test_Unet import tta_rolling
        hp_results = tta_rolling(model, channels, batch_size, ImgsegmentSize, NumsInputChannel, NumsClass, tta, ttalist, ttalistprob, deepsupervision)

        ## use the mask to constratin the results
        if os.path.isfile(DatafiletsMask):
            PredSegmentationWithinRoi = hp_results * roi_mask
        else:
            PredSegmentationWithinRoi = hp_results
        # PredSegmentationWithinRoi = predSegmentation

        predlabel = np.argmax(PredSegmentationWithinRoi, axis=0)
        
        labelwt = gtlabel == 1
        predwt = predlabel == 1

        DSCwt, SENSwt, PRECwt = ComputMetric(labelwt, predwt)

        # print(DSCwt)
        DSClist.append([DSCwt])
        
        for kcls in range(PredSegmentationWithinRoi.shape[0]):

            imgToSave = PredSegmentationWithinRoi[kcls, :, :, :]

            if saveresults:
                npDtype = np.dtype(np.float32)
                hdr_origin = Imgloadc1.header
                affine_origin = Imgloadc1.affine
                
                newLabelImg = nib.Nifti1Image(imgToSave, affine_origin)
                newLabelImg.set_data_dtype(npDtype)

                dimsImgToSave = len(imgToSave.shape)
                newZooms = list(hdr_origin.get_zooms()[:dimsImgToSave])
                if len(newZooms) < dimsImgToSave:  # Eg if original image was 3D, but I need to save a multi-channel image.
                    newZooms = newZooms + [1.0] * (dimsImgToSave - len(newZooms))
                newLabelImg.header.set_zooms(newZooms)

                directory = "../output/%s/%s/" % (pathname, name)
                if not os.path.exists(directory):
                    os.makedirs(directory)
                savename = directory + 'pred_' + kname + 'cls' + str(kcls) + '_prob.nii.gz'
                nib.save(newLabelImg, savename)
    
    DSCmean = np.array(DSClist)
    DSCmean = DSCmean.mean(axis=0)
    print(DSCmean)

In [174]:
testmap(model, args['saveresults'], args['name'] + '/results/', pathname=pathname,
                    ImgsegmentSize=args['patch_size'], deepsupervision=args['deepsupervision'],
                    DatafileValFoldtr=DatafileValFoldtr, DatafileValFoldts=DatafileValFoldts, NumsClass=NumsClass,
                    ttalist=args['ttalist'], ttalistprob=args['ttalistprob'])

case 0 done
case 1 done
case 2 done
case 3 done
case 4 done
case 5 done
case 6 done
case 7 done
case 8 done
case 9 done
[0.89257949]


In [184]:
## test condition x 83
for k in tqdm(range(1,84)):
    savename = 'prostattestcondition_' + str(k) 
    testlist = [k]
    args = {'resume': savedckpt, 'name': savename, 'saveresults': True, 'patch_size': patch_size, 'downsampling': 4,
       'ttalist': testlist, 'ttalistprob': testprob, 'features': 10, 'deepsupervision': True, 'gpu': 1}
    testmap(model, args['saveresults'], args['name'] + '/results/', pathname=pathname,
                    ImgsegmentSize=args['patch_size'], deepsupervision=args['deepsupervision'],
                    DatafileValFoldtr=DatafileValFoldtr, DatafileValFoldts=DatafileValFoldts, NumsClass=NumsClass,
                    ttalist=args['ttalist'], ttalistprob=args['ttalistprob'])

  1%|▏                 | 1/83 [01:06<1:31:02, 66.61s/it]

[0.8457301]



  2%|▍                 | 2/83 [02:13<1:30:27, 67.00s/it]

[0.87085208]



  4%|▋                 | 3/83 [03:27<1:33:26, 70.08s/it]

...

[0.85207284]



 99%|████████████████▊| 82/83 [1:09:01<00:33, 33.36s/it]

[0.85847882]


100%|█████████████████| 83/83 [1:09:32<00:00, 50.28s/it]

[0.79899082]


In [238]:
# test on 5 other domains
for prostatetestpath in tqdm(prostatetestpaths):
    DatafileValFoldts = prostatetestpath
    savename = 'prostattestcondition_' + prostatetestpath[-2:-1]
    args = {'resume': savedckpt, 'name': savename, 'saveresults': True, 'patch_size': patch_size, 'downsampling': 4,
       'ttalist': testlist, 'ttalistprob': testprob, 'features': 10, 'deepsupervision': True, 'gpu': 1}
    testmap(model, args['saveresults'], args['name'] + '/results/', pathname=pathname,
                    ImgsegmentSize=args['patch_size'], deepsupervision=args['deepsupervision'],
                    DatafileValFoldtr=DatafileValFoldtr, DatafileValFoldts=DatafileValFoldts, NumsClass=NumsClass,
                    ttalist=args['ttalist'], ttalistprob=args['ttalistprob'])


 20%|████                | 1/5 [01:47<07:10, 107.64s/it]

[0.66435248]



 40%|████████▍            | 2/5 [03:04<04:27, 89.26s/it]

[0.63897489]



 60%|████████████▌        | 3/5 [04:01<02:29, 74.55s/it]

[0.69487876]



 80%|████████████████▊    | 4/5 [04:39<01:00, 60.22s/it]

[0.75565781]


100%|█████████████████████| 5/5 [05:37<00:00, 67.55s/it]

[0.76435742]


In [185]:
# here for atlas, test-time operations

In [226]:
savename = 'atlasval'
savedckpt = atlasckpt
patch_size = [64, 64, 64]
testlist = [0]
testprob = [1]
NumsInputChannel = 1
NumsClass = 2
pathname = 'atlas'
DatafileValFoldtr = None
DatafileValFoldts = atlasvalpath

In [227]:
args = {'resume': savedckpt, 'name': savename, 'saveresults': True, 'patch_size': patch_size, 'downsampling': 4,
       'ttalist': testlist, 'ttalistprob': testprob, 'features': 10, 'deepsupervision': True, 'gpu': 0}

In [228]:
## test condition x 83
for k in tqdm(range(1,84)):
    savename = 'atlastestcondition_' + str(k) 
    testlist = [k]
    args = {'resume': savedckpt, 'name': savename, 'saveresults': True, 'patch_size': patch_size, 'downsampling': 4,
       'ttalist': testlist, 'ttalistprob': testprob, 'features': 10, 'deepsupervision': True, 'gpu': 1}
    testmap(model, args['saveresults'], args['name'] + '/results/', pathname=pathname,
                    ImgsegmentSize=args['patch_size'], deepsupervision=args['deepsupervision'],
                    DatafileValFoldtr=DatafileValFoldtr, DatafileValFoldts=DatafileValFoldts, NumsClass=NumsClass,
                    ttalist=args['ttalist'], ttalistprob=args['ttalistprob'])

  1%|▏              | 1/83 [19:47<27:03:20, 1187.81s/it]

[0.53707991]



  2%|▎              | 2/83 [40:08<27:09:41, 1207.18s/it]

[0.55419041]



  4%|▍            | 3/83 [1:02:29<28:11:00, 1268.25s/it]

[0.56657292]


  4%|▍            | 3/83 [1:16:56<34:11:51, 1538.90s/it]


KeyboardInterrupt: 